In [1]:
from elasticsearch import Elasticsearch
LOCAL = False
es_client = Elasticsearch(["elasticsearch"])

In [2]:
es_client.ping()

True

In [4]:
es_client.delete_by_query(index='_all', body={"query": {"match_all": {}}})

{'took': 10,
 'timed_out': False,
 'total': 0,
 'deleted': 0,
 'batches': 0,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

In [6]:
import json
import pandas as pd

In [7]:
with open("./scrapy_project/datatwo.json","r") as file:
    data = pd.read_json(file)
    
print(data[0:5])    
#data['votes'] = pd.to_numeric(data['votes'])
#data['answers'] = pd.to_numeric(data['answers'])
data['date'] = pd.to_datetime(data['date'])

len(data)

                                               title  votes  answers  \
0      How to call a method after a delay in Android    798       32   
1               Creating a textarea with auto-resize    394       44   
2  How to emulate GPS location in the Android Emu...    457       33   
3          What encoding/code page is cmd.exe using?    274        6   
4                  Escaping HTML strings with jQuery    626       26   

                       date   views  \
0 2010-06-18 18:24:19+00:00  690929   
1 2009-01-17 22:30:42+00:00  459837   
2 2010-02-17 09:55:51+00:00  419668   
3 2009-08-11 08:36:59+00:00  246499   
4 2008-08-24 02:52:42+00:00  682715   

                                                tags               author  
0                    [java, android, handler, delay]               aryaxt  
1       [javascript, html, resize, height, textarea]      DisgruntledGoat  
2  [android, testing, android-emulator, geolocation]  UMAR-MOBITSOLUTIONS  
3                  [windows,

298550

In [8]:
data = data.dropna()
data = data.to_dict('records')

In [9]:
data[0:5]

[{'title': 'How to call a method after a delay in Android',
  'votes': 798,
  'answers': 32,
  'date': Timestamp('2010-06-18 18:24:19+0000', tz='UTC'),
  'views': 690929,
  'tags': ['java', 'android', 'handler', 'delay'],
  'author': 'aryaxt'},
 {'title': 'Creating a textarea with auto-resize',
  'votes': 394,
  'answers': 44,
  'date': Timestamp('2009-01-17 22:30:42+0000', tz='UTC'),
  'views': 459837,
  'tags': ['javascript', 'html', 'resize', 'height', 'textarea'],
  'author': 'DisgruntledGoat'},
 {'title': 'How to emulate GPS location in the Android Emulator?',
  'votes': 457,
  'answers': 33,
  'date': Timestamp('2010-02-17 09:55:51+0000', tz='UTC'),
  'views': 419668,
  'tags': ['android', 'testing', 'android-emulator', 'geolocation'],
  'author': 'UMAR-MOBITSOLUTIONS'},
 {'title': 'What encoding/code page is cmd.exe using?',
  'votes': 274,
  'answers': 6,
  'date': Timestamp('2009-08-11 08:36:59+0000', tz='UTC'),
  'views': 246499,
  'tags': ['windows', 'command-line', 'encodin

In [10]:
from elasticsearch.helpers import bulk

In [11]:
def generate_data(documents):
    for docu in documents:
        yield {
            "_index": "stacks",
            "_type": "stack",
            "_source": {k:v if v else None for k,v in docu.items()},
        }

bulk(es_client, generate_data(data))

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/elasticsearch/connection/base.py:193: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


(292283, [])

In [3]:
!curl http://elasticsearch:9200/stacks/_search

{"took":1,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":10000,"relation":"gte"},"max_score":1.0,"hits":[{"_index":"stacks","_type":"stack","_id":"Kz77d3cBJXBYi4AvqWTl","_score":1.0,"_source":{"title":"How to call a method after a delay in Android","votes":798,"answers":32,"date":"2010-06-18T18:24:19+00:00","views":690929,"tags":["java","android","handler","delay"],"author":"aryaxt"}},{"_index":"stacks","_type":"stack","_id":"LD77d3cBJXBYi4AvqWTl","_score":1.0,"_source":{"title":"Creating a textarea with auto-resize","votes":394,"answers":44,"date":"2009-01-17T22:30:42+00:00","views":459837,"tags":["javascript","html","resize","height","textarea"],"author":"DisgruntledGoat"}},{"_index":"stacks","_type":"stack","_id":"LT77d3cBJXBYi4AvqWTl","_score":1.0,"_source":{"title":"How to emulate GPS location in the Android Emulator?","votes":457,"answers":33,"date":"2010-02-17T09:55:51+00:00","views":419668,"tags":["android","testing","andr

In [9]:
QUERY ="""{
    "mappings": {
        "type": {
          "properties": {
            "publisher": {
              "type": "text",
              "fielddata": true
            }
          }
        }
      }
}"""

In [11]:
!curl -XPUT http://elasticsearch:9200/stacks -d 
"""{
    "mappings": {
        "type": {
          "properties": {
            "publisher": {
              "type": "text",
              "fielddata": true
            }
          }
        }
      }
}"""

curl: option -d: requires parameter
curl: try 'curl --help' or 'curl --manual' for more information


'{\n    "mappings": {\n        "type": {\n          "properties": {\n            "publisher": {\n              "type": "text",\n              "fielddata": true\n            }\n          }\n        }\n      }\n}'

In [ ]:
result = es_client.search(index="stacks", body={"query": {"match_all": {}}})
result

In [ ]:
ids = []
for hit in result['hits']['hits']:
    #print("Name : {title}".format(**hit['_source']))
    #print("******************")
    ids.append("{title}".format(**hit['_source']))
print(ids[0])

In [ ]:
QUERY = {
  "query": {
    "multi_match" : {
      "query":    "stacks",
      "fields": [ "title", "views" ] 
    }
  },
    "sort" : [
        { "views" : {"order" : "desc"}}
    ]
}

In [ ]:
QUERY = {
  "query": {
    "term": {
      "title": "javascript"
    }
  }
}

In [ ]:
result = es_client.search(index="stacks", body=QUERY)
#[elt['_source']['title'] for elt in result["hits"]["hits"]]
result

In [ ]:
result['hits']['hits']

In [ ]:
ids = []
for hit in result['hits']['hits']:
    print("Name : {title}".format(**hit['_source']))
    print("******************")
    ids.append(hit["_id"])

In [20]:
es = Elasticsearch(["elasticsearch"])

In [21]:
!curl http://elasticsearch:9200/stacks/_search

{"took":1030,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":10000,"relation":"gte"},"max_score":1.0,"hits":[{"_index":"stacks","_type":"stack","_id":"mw4JY3cBFHzSIPGv9kTR","_score":1.0,"_source":{"title":"How to call a method after a delay in Android","votes":"798","answers":"32","date":"2010-06-18 18:24:19Z","views":690929,"tags":["java","android","handler","delay"],"author":"aryaxt"}},{"_index":"stacks","_type":"stack","_id":"nA4JY3cBFHzSIPGv9kTR","_score":1.0,"_source":{"title":"Creating a textarea with auto-resize","votes":"394","answers":"44","date":"2009-01-17 22:30:42Z","views":459837,"tags":["javascript","html","resize","height","textarea"],"author":"DisgruntledGoat"}},{"_index":"stacks","_type":"stack","_id":"nQ4JY3cBFHzSIPGv9kTR","_score":1.0,"_source":{"title":"How to emulate GPS location in the Android Emulator?","votes":"457","answers":"33","date":"2010-02-17 09:55:51Z","views":419668,"tags":["android","testing","andr